## WELCOME TO WEEK 6

The Epic Finale Week

And

## WELCOME TO

# MCP

And welcome back to OpenAI Agents SDK ❤️❤️❤️

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params) as server:
    fetch_tools = await server.list_tools()

for tool in fetch_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")

fetch: Fetches a URL from the internet and optionally extracts its contents as markdown.  Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.


### And now repeat for 3 more!

In [4]:
puppeteer_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-puppeteer"]}

async with MCPServerStdio(params=puppeteer_params) as server:
    puppeteer_tools = await server.list_tools()

for tool in puppeteer_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")


puppeteer_navigate: Navigate to a URL
puppeteer_screenshot: Take a screenshot of the current page or a specific element
puppeteer_click: Click an element on the page
puppeteer_fill: Fill out an input field
puppeteer_select: Select an element on the page with Select tag
puppeteer_hover: Hover an element on the page
puppeteer_evaluate: Execute JavaScript in the browser console


In [5]:
puppeteer_tools[0]

Tool(name='puppeteer_navigate', description='Navigate to a URL', inputSchema={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'URL to navigate to'}, 'launchOptions': {'type': 'object', 'description': "PuppeteerJS LaunchOptions. Default null. If changed and not null, browser restarts. Example: { headless: true, args: ['--no-sandbox'] }"}, 'allowDangerous': {'type': 'boolean', 'description': 'Allow dangerous LaunchOptions that reduce security. When false, dangerous args like --no-sandbox will throw errors. Default false.'}}, 'required': ['url']})

In [6]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params) as server:
    file_tools = await server.list_tools()

for tool in file_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")


read_file: Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.
read_multiple_files: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.
write_file: Create a new file or completely overwrite an existing file with new content. Use with caution as it will overwrite existing files without warning. Handles text content with proper encoding. Only works within allowed directories.
edit_file: Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showin

In [7]:

playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params) as server:
    playwright_tools = await server.list_tools()

for tool in playwright_tools:
    print(f"{tool.name.replace('\n', ' ')}: {tool.description.replace('\n', ' ')}")


browser_close: Close the page
browser_wait: Wait for a specified time in seconds
browser_file_upload: Upload one or multiple files
browser_install: Install the browser specified in the config. Call this if you get an error about the browser not being installed.
browser_press_key: Press a key on the keyboard
browser_navigate: Navigate to a URL
browser_navigate_back: Go back to the previous page
browser_navigate_forward: Go forward to the next page
browser_pdf_save: Save page as PDF
browser_snapshot: Capture accessibility snapshot of the current page, this is better than screenshot
browser_click: Perform click on a web page
browser_drag: Perform drag and drop between two elements
browser_hover: Hover over element on page
browser_type: Type text into editable element
browser_select_option: Select an option in a dropdown
browser_take_screenshot: Take a screenshot of the current page. You can't perform actions based on the screenshot, use browser_snapshot for actions.
browser_tab_list: List

### And now.. bring on the Agent with Tools!

In [8]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, cache_tools_list=True) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, cache_tools_list=True) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4o-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)


I've created a markdown file for the Banoffee Pie recipe. Here's a summary of the recipe:

```markdown
# Banoffee Pie Recipe

An easy and delicious Banoffee pie with a buttery pastry, rich dulce de leche, ripe bananas, and whipped cream.

## Ingredients
### For the Pastry:
- 100g butter (chilled, plus extra for greasing)
- 200g plain flour
- 1 medium egg (separated)
- 1 tbsp golden caster sugar

### For the Filling:
- 4 sliced bananas
- 394g caramel (or dulce de leche)
- 300ml double cream
- Dark chocolate (for garnish, optional)

## Method
1. **Make the Pastry:**  Combine the butter and flour in a food processor until it resembles breadcrumbs. Add the egg yolk and sugar, then mix again. Gradually add cold water until the dough comes together.
2. **Chill the Dough:** Remove and knead gently into a ball. Wrap in cling film and chill for 30 minutes.
3. **Bake the Pastry:** Preheat oven to 190C/170C fan/gas 4. Grease a 23cm fluted tart tin, roll out the pastry, and line the tin. Snip off 

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



